# Обучения языковых моделей

В этом ноутбуке будет произведено обучение языковых моделей для модели итеративного исправления. Требуется обучить две модели:

1. Слева-направо
2. Справа-налево

В качестве обучающего корпуса будет взят фрагмент корпуса Тайга, а именно части из соц.сетей, новостных сайтов, субтитров, так как это должен быть достаточно близкий к изучаемому домен.

В качестве модели было решено взять KenLM.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import gc
import sys
import os
import string
import re
from collections import Counter
sys.path.append('..')

import dotenv
import numpy as np
import pandas as pd

import nltk

from IPython.display import display
from tqdm.notebook import tqdm

In [3]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/mrgeekman/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
PROJECT_PATH = os.path.join(os.path.abspath(''), os.pardir)
CONFIGS_PATH = os.path.join(PROJECT_PATH, 'src', 'configs')
os.environ['DP_PROJECT_PATH'] = PROJECT_PATH

## Данные

В качестве данных для обучения решено было взять фрагмент корпуса [Тайга](https://tatianashavrina.github.io/taiga_site/). Были выбраны разделы:
1. Новости
2. Соцсети
3. Субтитры

Все файлы для скачивания доступны по [ссылке](https://tatianashavrina.github.io/taiga_site/downloads) в разделе "Our special collections for".

## Подготовка

В первую очередь требуется предобработать все тексты, что у нас имеются. Согласно задаче, нас не интересует регистр слов и пунктуация, поэтому избавимся от нее. В качестве результата должны получиться два текстовых файла (прямой и обратный), где каждое предложение расположено на отдельной строчке -- именно в таком виде следует подавать данные для обучающей программы.

In [5]:
!mkdir ../data/processed/kenlm -p

In [6]:
DATA_PATH = os.path.join(PROJECT_PATH, 'data')
TAIGA_PATH = os.path.join(DATA_PATH, 'external', 'taiga')
RESULT_PATH = os.path.join(DATA_PATH, 'processed', 'kenlm')
MODEL_PATH = os.path.join(PROJECT_PATH, 'models')

### Новости

Новости уже хорошо предобработаны. В каждом файле из `texts_tagged` уже содержатся тексты новостей под аттрибутом `text`, потребуется лишь их достать, проигнорировав первую строку.

In [46]:
def get_sentences(lines):
    """Нахождение предложений по строчкам в texts_tagged."""
    sentences = []
    for line in lines[1:]:
        if line.startswith('# text = '):
            sentences.append(line[len('# text = '):].strip().lower())
    return sentences

In [47]:
NEWS_PATH = os.path.join(TAIGA_PATH, 'news')
news_sentences = []

Сначала добавим только Фонтанку, потому что там есть деление по годам.

In [48]:
fontanka_path = os.path.join(NEWS_PATH, 'Fontanka', 'texts_tagged')
for year in tqdm(sorted(os.listdir(fontanka_path))):
    year_path = os.path.join(fontanka_path, year)
    for filename in sorted(os.listdir(year_path)):
        with open(os.path.join(year_path, filename), 'r') as inf:
            news_sentences += get_sentences(inf.readlines())

Теперь добавим предложения по всем остальным новостным сайтам.

In [54]:
for source in tqdm(sorted(os.listdir(NEWS_PATH))):
    if source == 'Fontanka':
        continue
    texts_path = os.path.join(NEWS_PATH, source, 'texts_tagged')
    for filename in sorted(os.listdir(texts_path)):
        with open(os.path.join(texts_path, filename), 'r') as inf:
            news_sentences += get_sentences(inf.readlines())

In [56]:
len(news_sentences)

4894406

Теперь каждый текст разобьем на токены и избавимся от тех из них, которые отвечают за пунктуацию. Сначала надо изучить какие вообще символы встречаются в текстах, чтобы понять что из этого может быть пунктуацией (иначе мы можем не учесть какие-то специфичные символы).

In [57]:
characters = Counter()
for sentence in tqdm(news_sentences):
    characters.update(list(sentence))

In [58]:
characters

Counter({'"': 3733357,
         'г': 8301552,
         'а': 39054989,
         'з': 7456501,
         'п': 14871033,
         'р': 26933314,
         'о': 52170857,
         'м': 14250265,
         ' ': 74609033,
         'и': 37013519,
         'б': 8034209,
         'е': 39736435,
         'л': 19787078,
         'у': 11955728,
         'с': 27138786,
         'я': 8533973,
         'д': 14686825,
         'ш': 2729334,
         'н': 31798432,
         'т': 30157793,
         'в': 22437980,
         'к': 16618999,
         'х': 3894825,
         'й': 5863506,
         '.': 5566502,
         ',': 6749140,
         'щ': 1938653,
         'ж': 4027292,
         'ц': 2890879,
         '1': 1713544,
         '0': 1944453,
         'ь': 6355169,
         'ч': 5746368,
         'ы': 8052626,
         'ф': 1956865,
         '2': 1569821,
         '7': 451294,
         '5': 620518,
         '4': 552888,
         '%': 105377,
         '3': 728700,
         '-': 2998758,
         'n': 551770,
 

Наличие иностранных символов можно объяснить ссылкой на какой-то иностранный источник или имя на оригинальном языке.

Посмотрим, какие символы мы уже имеем в пунктуации:

In [62]:
punctuation = string.punctuation
punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

Расширять его не требуется.

Будем удалять те токены, которые состоят лишь из знаков пунктуации. Запишем результаты на диск.

In [63]:
for sentence in tqdm(news_sentences):
    tokenized_sentence = nltk.tokenize.word_tokenize(
        sentence, language='russian'
    )
    cleaned_tokenized_sentence = [
        x for x in tokenized_sentence 
        if not re.fullmatch('[' + punctuation + ']+', x)
    ]
    
    with open(os.path.join(RESULT_PATH, 'news_left_right.txt'), 'a') as ouf:
        ouf.write(' '.join(cleaned_tokenized_sentence) + '\n')
        
    with open(os.path.join(RESULT_PATH, 'news_right_left.txt'), 'a') as ouf:
        ouf.write(' '.join(cleaned_tokenized_sentence[::-1]) + '\n')

In [64]:
del news_sentences
gc.collect()

1415

### Соцсети

Теперь обработаем тексты из соцсетей. Насчет включения этого раздела я до сих пор сомневаюсь. Тут весьма специфичный вокабуляр и достаточно много опечаток самих по себе.

In [70]:
SOCIAL_PATH = os.path.join(TAIGA_PATH, 'social', 'tagged_texts')
social_sentences = []

In [71]:
for source in sorted(os.listdir(SOCIAL_PATH)):
    with open(os.path.join(SOCIAL_PATH, source), 'r') as inf:
        social_sentences += get_sentences(inf.readlines())

In [72]:
len(social_sentences)

3215928

Осталось токенизировать предложения и выполнить запись на диск.

In [73]:
for sentence in tqdm(social_sentences):
    tokenized_sentence = nltk.tokenize.word_tokenize(
        sentence, language='russian'
    )
    cleaned_tokenized_sentence = [
        x for x in tokenized_sentence 
        if not re.fullmatch('[' + punctuation + ']+', x)
    ]
    
    with open(os.path.join(RESULT_PATH, 'social_left_right.txt'), 'a') as ouf:
        ouf.write(' '.join(cleaned_tokenized_sentence) + '\n')
        
    with open(os.path.join(RESULT_PATH, 'social_right_left.txt'), 'a') as ouf:
        ouf.write(' '.join(cleaned_tokenized_sentence[::-1]) + '\n')

In [74]:
del social_sentences
gc.collect()

1383

### Субтитры

Обработаем тексты из субтитров.

Особенность обработки в том, что в данных помимо текста есть таймкоды. Также одно и то же предложение в общем случае разбито на несколько таймкодов. Поэтому придется научиться фильтровать таймкоды при помощи регулярных выражений.

Начнем с того, что загрузим таблицу с метаданными, чтобы доставать файлы с русскими субтитрами.

In [104]:
SUBTITLES_PATH = os.path.join(TAIGA_PATH, 'subtitles')
subtitles_sentences = []

In [105]:
subtitles_df = pd.read_csv(os.path.join(SUBTITLES_PATH, 'metatable.csv'), sep='\t')
subtitles_df.head()

,id,title,languages,filepath
0,0,10 Things I Hate About You - 1x01 - Pilot.HDTV...,en,10 Things I Hate About You - 1x01 - Pilot.HDTV...
1,1,10 Things I Hate About You - 1x01 - Pilot.HDTV...,i,10 Things I Hate About You - 1x01 - Pilot.HDTV...
2,2,10 Things I Hate About You - 1x01 - Pilot.HDTV...,ru,10 Things I Hate About You - 1x01 - Pilot.HDTV...
3,3,10 Things I Hate About You - 1x02 - I Want You...,en,10 Things I Hate About You - 1x02 - I Want You...
4,4,10 Things I Hate About You - 1x02 - I Want You...,ru,10 Things I Hate About You - 1x02 - I Want You...


In [106]:
subtitles_df = subtitles_df[subtitles_df['languages'] == 'ru']
subtitles_df.head()

,id,title,languages,filepath
2,2,10 Things I Hate About You - 1x01 - Pilot.HDTV...,ru,10 Things I Hate About You - 1x01 - Pilot.HDTV...
4,4,10 Things I Hate About You - 1x02 - I Want You...,ru,10 Things I Hate About You - 1x02 - I Want You...
7,7,10 Things I Hate About You - 1x03 - Won't Get ...,ru,10 Things I Hate About You - 1x03 - Won't Get ...
10,10,10 Things I Hate About You - 1x04 - Don't Give...,ru,10 Things I Hate About You - 1x04 - Don't Give...
13,13,10 Things I Hate About You - 1x05 - Don't Give...,ru,10 Things I Hate About You - 1x05 - Don't Give...


Надо отдельно обработать случай сериала `Marvels Agents of S.H.I.E.L.D`. Дело в том, что данные между названиями второго и первого сезонов неконсистентны и это не полностью отражено в таблице (есть вариант написания `Marvel s Agents of S.H.I.E.L.D`).

In [107]:
subtitles_df[subtitles_df['filepath'].str.startswith('Marvel')].head()

,id,title,languages,filepath
11321,11321,Marvel s Agents of S.H.I.E.L.D. - 2x04 - Face ...,ru,Marvel s Agents of S.H.I.E.L.D. - 2x04 - Face ...
11325,11325,Marvel s Agents of S.H.I.E.L.D. - 2x11 - After...,ru,Marvel s Agents of S.H.I.E.L.D. - 2x11 - After...
11327,11327,Marvels Agents of S.H.I.E.L.D. - 1x01 - Pilot....,ru,Marvels Agents of S.H.I.E.L.D. - 1x01 - Pilot....
11328,11328,Marvels Agents of S.H.I.E.L.D. - 1x02 - 0-8-4....,ru,Marvels Agents of S.H.I.E.L.D. - 1x02 - 0-8-4....
11330,11330,Marvels Agents of S.H.I.E.L.D. - 1x03 - The As...,ru,Marvels Agents of S.H.I.E.L.D. - 1x03 - The As...


In [108]:
filenames = subtitles_df['filepath'].tolist()
folders = [x.split(' - ')[0].strip(' .').replace('Marvels', 'Marvel s') for x in filenames]

In [109]:
for folder, filename in tqdm(zip(folders, filenames), total=len(folders)):
    with open(os.path.join(SUBTITLES_PATH, 'tagged_texts', folder, filename), 'r') as inf:
        subtitles_sentences += get_sentences(inf.readlines())

In [110]:
len(subtitles_sentences)

3948916

Теперь уберем таймкоды из предложений. Для этого воспользуемся регулярными выражениями.

In [127]:
example = subtitles_sentences[11]
example

'тебе ясно, бьянка 11 00:00:32,610 00:00:36,437 но они не сделают этого, потому что ты запретил мне с ними встречаться, пока кэт не начнет.'

In [128]:
re.sub('\d+ \d\d:\d\d:\d\d,\d\d\d \d\d:\d\d:\d\d,\d\d\d', '', example)

'тебе ясно, бьянка  но они не сделают этого, потому что ты запретил мне с ними встречаться, пока кэт не начнет.'

In [129]:
subtitles_sentences = [
    re.sub('\d+ \d\d:\d\d:\d\d,\d\d\d \d\d:\d\d:\d\d,\d\d\d', '', x).strip()
    for x in subtitles_sentences
]

Осталось токенизировать предложения и выполнить запись на диск. Также следует очистить 

In [130]:
lengths = []
for sentence in tqdm(subtitles_sentences):
    tokenized_sentence = nltk.tokenize.word_tokenize(
        sentence.replace('\t', ' ').replace('\n', ' '), language='russian'
    )
    cleaned_tokenized_sentence = [
        x for x in tokenized_sentence 
        if not re.fullmatch('[' + punctuation + ']+', x)
    ]
    with open(os.path.join(RESULT_PATH, 'subtitles_left_right.txt'), 'a') as ouf:
        ouf.write(' '.join(cleaned_tokenized_sentence) + '\n')
        
    with open(os.path.join(RESULT_PATH, 'subtitles_right_left.txt'), 'a') as ouf:
        ouf.write(' '.join(cleaned_tokenized_sentence[::-1]) + '\n')

In [131]:
del subtitles_sentences, subtitles_df
gc.collect()

1561

### Сборка обучающего датасета

Теперь сконкатенируем полученные файлы для обучения языковых моделей.

In [132]:
!cat ../data/processed/kenlm/news_left_right.txt ../data/processed/kenlm/social_left_right.txt ../data/processed/kenlm/subtitles_left_right.txt > ../data/processed/kenlm/left_right.txt

In [133]:
!cat ../data/processed/kenlm/news_right_left.txt ../data/processed/kenlm/social_right_left.txt ../data/processed/kenlm/subtitles_right_left.txt > ../data/processed/kenlm/right_left.txt

Посмотрим на объем полученных датасетов.

In [134]:
!du ../data/processed/kenlm/left_right.txt -h

1.7G	../data/processed/kenlm/left_right.txt


In [135]:
!du ../data/processed/kenlm/right_left.txt -h

1.7G	../data/processed/kenlm/right_left.txt


## Обучение

Теперь выполним обучение. Для этого вспользуемя [документацией](https://kheafield.com/code/kenlm/estimation/) и [инструкцией](https://github.com/kmario23/KenLM-training).

На этом этапе подразумевается, что библиотека уже склонирована в src/kenlm и собрана.

In [136]:
! ../src/kenlm/build/bin/lmplz -o 3 < ../data/processed/kenlm/left_right.txt > ../models/kenlm/left_right.arpa

=== 1/5 Counting and sorting n-grams ===
Reading /home/mrgeekman/Documents/MIPT/НИР/Repo/data/processed/kenlm/left_right.txt
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
Unigram tokens 145379227 types 2168057
=== 2/5 Calculating and sorting adjusted counts ===
Chain sizes: 1:26016684 2:3399487488 3:6374039040
Statistics:
1 2168057 D1=0.752992 D2=0.924691 D3+=1.14515
2 30503408 D1=0.796475 D2=1.08615 D3+=1.3121
3 73612195 D1=0.83601 D2=0.766706 D3+=0.020062
Memory estimate for binary LM:
type      MB
probing 2015 assuming -p 1.5
probing 2198 assuming -r models -p 1.5
trie     921 without quantization
trie     549 assuming -q 8 -b 8 quantization 
trie     853 assuming -a 22 array pointer compression
trie     480 assuming -a 22 -q 8 -b 8 array pointer compression and quantization
=== 3/5 Calculating and sorting initial probabilities =

In [137]:
! ../src/kenlm/build/bin/lmplz -o 3 < ../data/processed/kenlm/right_left.txt > ../models/kenlm/right_left.arpa

=== 1/5 Counting and sorting n-grams ===
Reading /home/mrgeekman/Documents/MIPT/НИР/Repo/data/processed/kenlm/right_left.txt
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
Unigram tokens 145379227 types 2168057
=== 2/5 Calculating and sorting adjusted counts ===
Chain sizes: 1:26016684 2:3399487488 3:6374039040
Statistics:
1 2168057 D1=0.750457 D2=0.911682 D3+=1.17527
2 30503408 D1=0.79503 D2=1.07301 D3+=1.29189
3 73612195 D1=0.83601 D2=0.766706 D3+=0.020062
Memory estimate for binary LM:
type      MB
probing 2015 assuming -p 1.5
probing 2198 assuming -r models -p 1.5
trie     921 without quantization
trie     549 assuming -q 8 -b 8 quantization 
trie     853 assuming -a 22 array pointer compression
trie     480 assuming -a 22 -q 8 -b 8 array pointer compression and quantization
=== 3/5 Calculating and sorting initial probabilities =

### Бинаризация

Бинаризуем модель, чтобы ей можно было быстрее пользоваться.

In [138]:
!../src/kenlm/build/bin/build_binary ../models/kenlm/left_right.arpa ../models/kenlm/left_right.arpa.binary

Reading ../models/kenlm/left_right.arpa
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
SUCCESS


In [139]:
!../src/kenlm/build/bin/build_binary ../models/kenlm/right_left.arpa ../models/kenlm/right_left.arpa.binary

Reading ../models/kenlm/right_left.arpa
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
SUCCESS


Удалим теперь небинаризованные модели.

In [140]:
!rm ../models/kenlm/left_right.arpa
!rm ../models/kenlm/right_left.arpa

## Тест

А теперь загрузим модель и попробуем применить ее к какому-либо предложению.

In [141]:
import kenlm

model_left_right = kenlm.LanguageModel(os.path.join(MODEL_PATH, 'kenlm', 'left_right.arpa.binary'))
model_right_left = kenlm.LanguageModel(os.path.join(MODEL_PATH, 'kenlm', 'right_left.arpa.binary'))

In [142]:
example = 'журналисты всегда все нагло беспардонно переврут'
example_reversed = ' '.join(example.split(' ')[::-1])

In [143]:
model_left_right.score(example)

-29.408767700195312

In [144]:
model_right_left.score(example_reversed)

-29.43677520751953